# homuncu_loc image viewer

In [7]:
import napari
from skimage import io
import btrack
import os
from homuncu_loc import dataio
from glob import glob

### Define base directory and image ID

In [14]:
img_base_dir = '/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_experiments/DAPI_NKX21_PDPN_ZO1/images'
image_base_name = '20x_21-12-028A_A23456_Multichannel Z-Stack_20220818_237.tif' 
image_fn = os.path.join(img_base_dir, image_base_name)
image_id = dataio.ID_extractor(image_fn)
print(f"File '{image_base_name}' {'exists.' if os.path.exists(image_fn) else 'does not exist.'}")

File '20x_21-12-028A_A23456_Multichannel Z-Stack_20220818_237.tif' exists.


### Load image

In [ ]:
image = io.imread(image_fn)
print(f'Image of shape {image.shape} loaded')

### Search for associated single-cell segmentation and tracking data

In [9]:
# base_dir = '/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_experiments/DAPI_NKX21_PDPN_ZO1/sc_analyses'
sc_base_dir = img_base_dir.replace('images', 'sc_analyses') 
sc_fns = glob(os.path.join(sc_base_dir, f'*_{image_id}*.h5'))
print(sc_fns)

['/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_experiments/DAPI_NKX21_PDPN_ZO1/sc_analyses/20x_21-12-028A_A23456_Multichannel Z-Stack_20220818_237_iat2.h5']


### Load single-cell data

In [10]:
sc_fn = sc_fns[0]
# load labels (tracks and segmentation)
with btrack.io.HDF5FileHandler(sc_fn, 
                                   'r', 
                                       obj_type='obj_type_2'#'obj_type_2'
                                   ) as reader:
        # tracks = [t for t in reader.tracks if np.mean(t.properties['area']) > 1000]
        tracks = [t for t in reader.tracks if len(t) >= 3]
        segmentation = reader.segmentation
napari_tracks, properties, graph = btrack.utils.tracks_to_napari(tracks, ndim = 2)

[INFO][2023/10/16 11:06:00 AM] Opening HDF file: /run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_experiments/DAPI_NKX21_PDPN_ZO1/sc_analyses/20x_21-12-028A_A23456_Multichannel Z-Stack_20220818_237_iat2.h5...
[INFO][2023/10/16 11:06:00 AM] Loading tracks/obj_type_2
[INFO][2023/10/16 11:06:00 AM] Loading LBEP/obj_type_2
[INFO][2023/10/16 11:06:00 AM] Loading objects/obj_type_2 (28241, 5) (28241 filtered: None)
[INFO][2023/10/16 11:06:04 AM] Loading segmentation (81, 2304, 2304)
[INFO][2023/10/16 11:06:04 AM] Closing HDF file: /run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_experiments/DAPI_NKX21_PDPN_ZO1/sc_analyses/20x_21-12-028A_A23456_Multichannel Z-Stack_20220818_237_iat2.h5


##### Optionally recolour segmentation according to Cell ID

In [15]:
recolored_segmentation = btrack.utils.update_segmentation(segmentation, tracks, color_by='ID')

### View images and single-cell data

In [16]:
viewer = napari.Viewer()

viewer.add_image(image, channel_axis = -1)
# viewer.add_labels(segmentation, visible = False)
viewer.add_labels(recolored_segmentation)
viewer.add_tracks(napari_tracks)

<Tracks layer 'napari_tracks' at 0x7f4194b70d30>

In [20]:
viewer.add_image(segmentation)

<Image layer 'segmentation' at 0x7f3ffc1a96a0>

In [17]:
print()